<a href="https://colab.research.google.com/github/lkshiel/EPA_Projects/blob/main/RunPkasolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# @title Initializing Condacolab
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:

# @title Check that everything is fine

import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
# @title Installing dependencies and pkasolver package (this might take up to 20 minutes)

print('📦 Installing dependencies ...')
!mamba install -c conda-forge rdkit > /dev/null
print('🔥 Installing PyTorch and PyG ...')
!pip install torch==1.13.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html > /dev/null
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.1+cpu.html > /dev/null
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.1+cpu.html > /dev/null
!pip install torch-spline-conv torch-geometric==2.0.1 -f https://data.pyg.org/whl/torch-1.13.1+cpu.html > /dev/null
!pip install cairosvg svgutils molvs > /dev/null
print('✔️ Installing pkasolver package ...')
!pip install -q git+https://github.com/mayrf/pkasolver.git > /dev/null
print("🎉 Done!")

📦 Installing dependencies ...
🔥 Installing PyTorch and PyG ...
✔️ Installing pkasolver package ...
🎉 Done!


In [ ]:
# @title Deal with numpy issue
!pip uninstall numpy

!pip install numpy==1.26.4

Found existing installation: numpy 2.2.4
Uninstalling numpy-2.2.4:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/site-packages/numpy-2.2.4.dist-info/*
    /usr/local/lib/python3.11/site-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-2.2.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 77.8 MB/s eta 0:00:00


In [ ]:
#import dependancies
import pickle
import pandas as pd
from rdkit import Chem
import torch
from os import path

#import pkasolver
import pkasolver
from pkasolver.query import QueryModel
from pkasolver.ml_architecture import GINPairV1
from pkasolver.query import draw_pka_map
from pkasolver.query import calculate_microstate_pka_values, draw_pka_reactions
from IPython.display import display
from IPython.display import HTML

#load trained model
base_path=path.dirname(pkasolver.__file__)

import warnings
warnings.filterwarnings('ignore')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

### Generator function
Runs pkasolver through a generator function.

This prevents overloading the tmp pickle file and allowing for a bettter dealing of batch sample submission.

In [ ]:
def RunPkasolver(x): #takes input of smile object
    protonation_states = calculate_microstate_pka_values(x) #performs internal calculations and stores as object
    sites=len(protonation_states) #get the number of ionization sites

    lst=[]
    for j in range(len(protonation_states)):
        state=protonation_states[j]
        lst.append(round(state.pka,2)) #get pka values for all sites for a given molecule store in a list

    yield sites, lst

### Single chemical example run
runs a single smiles string

only need to edit the first line;
smiles='XXXXX'

where XXXXX is your smiles of interest

In [ ]:
smiles='OCC(F)(F)F'
mol=Chem.MolFromSmiles(smiles)
res=RunPkasolver(mol)
for n,p in res:
  print('pka:',p,'number of sites:',n)

[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


Proposed mol at pH 7.4: OCC(F)(F)F
pka: [9.49] number of sites: 1


### Batch example run

Data must be uploaded to google drive

path to file must be included at input

  my example: /content/drive/MyDrive/EPA

list of smiles strings can be imported as csv,xlsx,txt,or smi but the header MUST be 'SMILES'

depending on file type, the read-in code will be differnt.
Run the line of code that suites your input file type

Note: ./EPA/TestData.xlsx is an example and will not work for you because you do not have the folder or file.

In [ ]:
#access google drive
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
#navigate to file path, check that desired file is there
!ls /content/drive/MyDrive/EPA

TestData.xlsx


#### If data file is a csv...

In [ ]:
inpt=pd.read_csv('/content/drive/MyDrive/EPA/TestData.csv')

#### if data file is .smi or .txt

In [ ]:
inpt=pd.read_table('/content/drive/MyDrive/EPA/TestData.smi')

#### if data file is xlsx...

In [ ]:
inpt=pd.read_excel('/content/drive/MyDrive/EPA/TestData.xlsx')
inpt.head()

,Compound,DTXSID,SMILES
0,Acetic acid,DTXSID5024394,CC(O)=O
1,Phosphate,DTXSID5024263,OP(O)(O)=O
2,Methyl-Phosphate,DTXSID2025684,COP(O)(O)=O
3,Ethyl-Phosphate,DTXSID1044699,CCOP(O)(=O)OCC
4,Dimethyl-Phosphate,DTXSID5025150,COP(O)(=O)OC


In [ ]:
#make an empty results dataframe called results
results=pd.DataFrame(columns=['SMILES','sites_num','pka'])

#fill results dataframe with exisitng information
results['SMILES']=inpt['SMILES']
results

,SMILES,sites_num,pka
0,CC(O)=O,NaN,NaN
1,OP(O)(O)=O,NaN,NaN
2,COP(O)(O)=O,NaN,NaN
3,CCOP(O)(=O)OCC,NaN,NaN
4,COP(O)(=O)OC,NaN,NaN
5,CCC[NH3+],NaN,NaN
6,[NH3+]CCC[NH3+],NaN,NaN
7,[NH3+]CCCC(O)=O,NaN,NaN
8,[NH3+]CCCS(O)(=O)=O,NaN,NaN
9,CCCCN,NaN,NaN


In [ ]:
#loop through smiles input, calculate pka, and store in results table
for i in results.index:
    smi = inpt['SMILES'][i]
    mol = Chem.MolFromSmiles(smi) #convert string into smile object (rdkit)
    data = RunPkasolver(mol) # run smile object through generator function to get pka and num of ionionizable sites

    #sep out generator yield and save to results table
    for n,p in data:
        results['pka'][i]=p
        results['sites_num'][i]=n
results

[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


Proposed mol at pH 7.4: CC(=O)[O-]


[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


Proposed mol at pH 7.4: O=P([O-])([O-])[O-]


[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


...

[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


Proposed mol at pH 7.4: [NH3+]CCCS(=O)(=O)[O-]


[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


Proposed mol at pH 7.4: CCCC[NH3+]


,SMILES,sites_num,pka
0,CC(O)=O,1,[4.19]
1,OP(O)(O)=O,1,[2.83]
2,COP(O)(O)=O,1,[2.56]
3,CCOP(O)(=O)OCC,1,[4.66]
4,COP(O)(=O)OC,1,[4.06]
5,CCC[NH3+],1,[10.77]
6,[NH3+]CCC[NH3+],1,[11.19]
7,[NH3+]CCCC(O)=O,2,"[5.07, 10.61]"
8,[NH3+]CCCS(O)(=O)=O,1,[9.65]
9,CCCCN,1,[10.93]


### Save Results

In [ ]:
results.to_excel('/content/drive/My Drive/EPA/Results_TestData.xlsx',index=False)

In [ ]:
#check that results file is in your folder
!ls /content/drive/MyDrive/EPA

Results_TestData.xlsx  TestData.xlsx
